In [70]:
import yfinance as yf
import os
import pandas as pd
import ccxt
from datetime import datetime
from dateutil.relativedelta import relativedelta

**Загружаем данные акций из индекса S&P500**

In [71]:
finance_data = {}
tickers_sp500 = ['MMM', 'ADBE']

for ticker in tickers_sp500:
    ticker_data_file_name = os.path.join('data', f'{ticker}.csv')

    if not os.path.isfile(ticker_data_file_name):
        finance_data[ticker] = yf.Ticker(f'{ticker}').history(period='1y')
        finance_data[ticker].to_csv(ticker_data_file_name)
    else:
        finance_data[ticker] = pd.read_csv(ticker_data_file_name)


**Загружаем данные по криптовалютам**

In [72]:
tickers_crypto = ['BTC', 'ETH', 'SOL', 'XRP']
from_ts = datetime.now() - relativedelta(years=1)

exchange = ccxt.binance({
    'enableRateLimit': True,
})

since = exchange.parse8601(from_ts.strftime('%Y-%m-%d %H:%M:%S'))

for ticker in tickers_crypto:
    ticker_data_file_name = os.path.join('data', f'{ticker}.csv')

    if not os.path.isfile(ticker_data_file_name):
        ohlcv = exchange.fetch_ohlcv(f'{ticker}/USDT', '1d', since=since)
        temp_df = pd.DataFrame(ohlcv, columns=['Date', 'Open', 'Hign', 'Low', 'Close', 'Volume'])
        temp_df['Date'] = pd.to_datetime(temp_df['Date'], unit='ms')
        finance_data[ticker] = temp_df
        finance_data[ticker].to_csv(ticker_data_file_name)
    else:
        finance_data[ticker] = pd.read_csv(ticker_data_file_name)

**Находим максимальные цены закрытия**

In [73]:
for ticker in finance_data:
    max_close_price = finance_data[ticker]['Close'].max()
    print(f'The maximum close price of {ticker} is {max_close_price:.3f}')

The maximum close price of MMM is 134.690
The maximum close price of ADBE is 634.760
The maximum close price of BTC is 73072.410
The maximum close price of ETH is 4064.800
The maximum close price of SOL is 202.450
The maximum close price of XRP is 0.723
